# 인스타그램 데이터 추출

In [27]:
import re
import time
import pandas as pd
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver


def instagram_login(login_info):
    driver.get(base_url)
    time.sleep(3)
    
    input_id = driver.find_elements_by_class_name("_2hvTZ.pexuQ.zyHYP")[0]
    input_id.clear()
    input_id.send_keys(login_info[0])

    input_pw = driver.find_elements_by_class_name("_2hvTZ.pexuQ.zyHYP")[1]
    input_pw.clear()
    input_pw.send_keys(login_info[1])
    input_pw.submit()
    time.sleep(5)
    
    save_info_btn = driver.find_element_by_class_name("sqdOP.L3NKy.y3zKF")
    save_info_btn.click()
    time.sleep(5)
    
    notifications_btn = driver.find_element_by_class_name("aOOlW.HoLwm")
    notifications_btn.click()
    time.sleep(5)


def instagram_search(query):
#     query = input()
    search_url = base_url + "explore/tags/" + quote_plus(query)
    driver.get(search_url)
    time.sleep(5)


def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click() # 사진을 클릭한다
    time.sleep(2) # 사진이 로딩되는 시간동안 대기


def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    try:
        content = soup.select("div.C4VMK > span")[0].text
    except:
        content = ""
    
    # content에서 해시태그만 가져오기
    hashtag = re.findall("#[^\s]+", content)
    
    # 작성일자 가져오기
    date = soup.select("time._1o9PC.Nzb55")[0]['datetime']
    
    # 좋아요 수 가져오기
    
    like = ''
    if soup.select(".HbPOm._9Ytll"):
        like = soup.select(".HbPOm._9Ytll > span > span")[0].text
    elif soup.select("div.Nm9Fw > button > span"):
        like = soup.select("div.Nm9Fw > button > span")[0].text
    
    # 위치정보 가져오기
    location = soup.select("div.M30cS")[0].text
    
    data = [content, hashtag, date, like, location]
    return data
    

def moveNext(driver):
    time.sleep(2)
    right = driver.find_element_by_css_selector("a.coreSpriteRightPaginationArrow")
    right.click()
    time.sleep(2)
    

driver = webdriver.Chrome("/Users/yoon/.selenium-drivers/chromedriver")
base_url = "https://instagram.com/"

login_info = ["ehc.1010", "davidho1358"]
query = "제주도맛집"

instagram_login(login_info)
instagram_search(query)

select_first(driver)

results = []
for i in range(10):
    data = get_content(driver)
    results.append(data)
    moveNext(driver)
    
instagram_df = pd.DataFrame(results, columns = ["content", "hashtag", "data", "like", "location"])
instagram_df

,content,hashtag,data,like,location
0,안녕하세요^^이춘옥의 원조!원조! 고등어쌈밥입니다.가끔은 어머니가 가게를 작게 하셨...,"[#애월맛집, #제주도맛집#제주시맛집, #제주공항근처맛집#이호테우해변맛집, #제주애...",2020-08-09T01:36:29.000Z,"1,005",
1,.맛나는 음식보다만나는 음식이길 원해요..만남의 화려함보단작별의 그리움으로 남길 원...,[#월정리갈비밥.월정리점],2020-08-09T01:07:13.000Z,"2,291",
2,⠀제주애월 핵🔥맛집 #장인의집대형가마솥에 24시간 푹고은 한우사골과 제주바다 듬뿍담...,"[#장인의집대형가마솥에, #제주도맛집, #제주맛집, #제주애월맛집, #애월맛집, #...",2020-08-09T07:24:04.000Z,861,환상의 섬 Jeju Island
3,가성비 좋은 한끼 원하시는 잇님들!!⚈͒◡⚈͒혼술 혼밥 포장 🎁 다 가능합니다.⚈͒...,"[#섭지코지한끼🍱오픈, #성산맛집, #서귀포맛집, #섭지코지카페#성산일출봉, #서귀...",2020-08-09T17:37:47.000Z,183,
4,제주도 놀러왔는데 제대로 한끼 드셔야겠죠?섭지코지근처에 있는 #섭지코지맛집 !!!⠀...,"[#섭지코지맛집, #섭지코지한끼, #제주도맛집, #제주맛집#제주도카페, #제주카페#...",2020-08-09T15:47:02.000Z,278,
5,성산일출봉🏔 해안가🏖 바로 앞에 위치해 있고 하와이풍 인테리어와 다양한 해산물을 같...,"[#코코마마⠀#제주도맛집, #제주맛집#제주도카페, #제주카페#성산일출봉맛집, #성산...",2020-08-09T15:55:30.000Z,187,
6,백종원님이 방송에서 천상의맛이라고 극찬한 그 '카이막'👍🏻👍🏻#쉬어갓 오시면 드실...,"[#쉬어갓, #제주카페, #제주도맛집#제주맛집, #제주도카페#오설록맛집, #종달리맛...",2020-08-09T17:48:51.000Z,348,
7,친절한 사장님과 푸성한 구성 탓에 제주에 지인오면 항상 데려가는 곳 🤤_이번에 멜조...,[#제주도맛집#제주맛집#제주공항근처맛집#제주도공항근처맛집#제주도흑돼지#제주흑돼지#제...,2020-08-09T03:45:44.000Z,"1,421",제주육대표
8,가격 대비 푸짐한 구성으로 입소문이 자자한코코마마🦞🦞🦞.성산일출봉 해안가 바로 앞에...,"[#코코마마⠀#성산일출봉맛집, #서귀포맛집#비자림맛집, #우도맛집#제주맛집, #제주...",2020-08-09T17:34:09.000Z,211,
9,#어제이야기⠀⠀아침에 눈 뜨자마자 하늘을 보러 나가보니 햇님이⠀우리를 반겨주었어요~...,[#어제이야기⠀⠀아침에],2020-08-10T00:49:35.000Z,,


# 스타벅스 데이터 분석

* 스타벅스 홈페이지 -> 매장 정보 -> 크롤링  
    * http://www.istarbucks.co.kr
* 서울시 열린데이터 광장 -> 인구통계 데이터  
* open api(application programming interface):
    * 데이터 분석에서 데이터를 제공할 수 있도록 만든 인터페이스

In [46]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

browser = webdriver.Safari()
url = "https://www.starbucks.co.kr/store/store_map.do?disp=locale"
browser.get(url)
time.sleep(2)

seoulBtn = "#container > div > form > fieldset > div > section > article.find_store_cont \
            > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
browser.find_element_by_css_selector(seoulBtn).click()
time.sleep(2)

allBtn = "#mCSB_2_container > ul > li:nth-child(1) > a"
browser.find_element_by_css_selector(allBtn).click()
time.sleep(2)


In [257]:
soup = BeautifulSoup(browser.page_source, "lxml")
all_stores_info = []
for store in soup.select("#mCSB_3_container > ul > li"):
    all_stores_info.append([store['data-name'],
                            store.select('i')[0]['class'][0][4:],
                            str(store.findChild("br").nextSibling).strip(), 
                            store['data-lat'],
                            store['data-long'],
                            str(store.findChild("br").previousSibling).strip()])
    
starbucks_df = pd.DataFrame(all_stores_info, columns = ['STORE','STORE_TYPE', 'PHONE_NUMBER', 'LAT', 'LNG', 'ADDR'])
starbucks_df.head(10)

,STORE,STORE_TYPE,PHONE_NUMBER,LAT,LNG,ADDR
0,역삼아레나빌딩,general,1522-3232,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동)
1,논현역사거리,general,1522-3232,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동)
2,신사역성일빌딩,general,1522-3232,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동)
3,국기원사거리,general,1522-3232,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동)
4,스탈릿대치R,reserve,1522-3232,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동)
5,봉은사역,general,1522-3232,37.515000,127.063196,서울특별시 강남구 봉은사로 619 (삼성동)
6,압구정윤성빌딩,general,1522-3232,37.5227934,127.0286009,서울특별시 강남구 논현로 834 (신사동)
7,코엑스별마당,general,1522-3232,37.510150,127.060275,서울특별시 강남구 영동대로 513 (삼성동)
8,삼성역섬유센터R,reserve,1522-3232,37.507750,127.060651,서울특별시 강남구 테헤란로 518 (대치동)
9,압구정R,reserve,1522-3232,37.5273669,127.033061,서울특별시 강남구 언주로 861 (신사동)


In [143]:
starbucksStore = soup.select("#mCSB_3_container > ul > li")[0]

bs4.element.Tag

In [114]:
#storeName = starbucksStore.select('strong')[0].text.strip()
storeName = starbucksStore['data-name']

In [106]:
storeLat = starbucksStore['data-lat']
storeLong = starbucksStore['data-long']

In [137]:
storeAddress = str(starbucksStore.select('p')[0]).split("<br/>")[0].split(">")[1]
storePhoneNumber = str(starbucksStore.select('p')[0]).split("<br/>")[1].split("<")[0]

print(storeAddress, storePhoneNumber)

서울특별시 강남구 언주로 425 (역삼동) 1522-3232


In [103]:
print(starbucksStore.select('i'))
print(starbucksStore.select('i')[0])
print(starbucksStore.select('i')[0]['class'])
print(starbucksStore.select('i')[0]['class'][0])
print(starbucksStore.select('i')[0]['class'][0][4:])

storeType = starbucksStore.select('i')[0]['class'][0][4:] 

[<i class="pin_general">리저브 매장 2번</i>]
<i class="pin_general">리저브 매장 2번</i>
['pin_general']
pin_general
general


In [303]:
starbucks_df.to_excel("seoul_starbucks_list.xlsx", index=False)

# 서울시 데이터 불러오기
1. 서울열린데이터광장 인증키 발급
2. API 호출
    * WGS1984 or 서울시 행정구역 시군구 정보 -> Open API 주소 형식에 맞는 요청인자 사용
 
### 요청인자
|변수명|타입|변수설명|값설명|
|:-:|:-:|:-:|:-:|
|KEY|String(필수)|인증키|OpenAPI 에서 발급된 인증키|
|TYPE|String(필수)|요청파일타입|xml : xml, xml파일 : xmlf, 엑셀파일 : xls, json파일 : json|
|SERVICE|String(필수)|서비스명|SdeTlSccoSigW|
|START_INDEX|INTEGER(필수)|요청시작위치|정수 입력 (페이징 시작번호 입니다 : 데이터 행 시작번호)|
|END_INDEX|INTEGER(필수)|요청종료위치|정수 입력 (페이징 끝번호 입니다 : 데이터 행 끝번호)|
|OBJECTID|STRING(선택)|순번|정수 입력|



In [187]:
import requests


# 발급받은 인증키
SEOUL_API_AUTH_KEY = "6b51517250796f6f3533526b665263"

args = [SEOUL_API_AUTH_KEY, "json", "SdeTlSccoSigW", 1, 25, ""]
 #API 호출
url = "http://openapi.seoul.go.kr:8088/{}/{}/{}/{}/{}/{}".format(*args)
response = requests.get(url).json()

In [188]:
response_data = response["SdeTlSccoSigW"]
response_list = response_data['row']

response_df = pd.DataFrame(response_list)

In [189]:
response_df.head()

,OBJECTID,SIG_CD,SIG_KOR_NM,SIG_ENG_NM,ESRI_PK,LAT,LNG
0,1.0,11320,도봉구,Dobong-gu,0.0,37.6658609,127.0317674
1,2.0,11380,은평구,Eunpyeong-gu,1.0,37.6176125,126.9227004
2,3.0,11230,동대문구,Dongdaemun-gu,2.0,37.5838012,127.0507003
3,4.0,11590,동작구,Dongjak-gu,3.0,37.4965037,126.9443073
4,5.0,11545,금천구,Geumcheon-gu,4.0,37.4600969,126.9001546


### 일반화 함수를 이용하여 시군구 open api 호출

In [241]:
def seoul_open_api_data(url, url_args):
    try:
        response_dict = requests.get(url.format(*url_args)).json()
        response_data = response_dict[url_args[2]]
        
        code = response_data['RESULT']['CODE']
        
        if code == 'INFO-000':
            data_list = response_data['row']
            
        return data_list
    except:
        print("Invalid Key")
        
    return None

url = "http://openapi.seoul.go.kr:8088/{}/{}/{}/{}/{}/{}"
url_args = [SEOUL_API_AUTH_KEY, "json", "SdeTlSccoSigW", 1, 25, ""]

seoul_sgg_list = seoul_open_api_data(url, url_args)
seoul_sgg_df = pd.DataFrame(seoul_sgg_list)

seoul_sgg_df.head()

,OBJECTID,SIG_CD,SIG_KOR_NM,SIG_ENG_NM,ESRI_PK,LAT,LNG
0,1.0,11320,도봉구,Dobong-gu,0.0,37.6658609,127.0317674
1,2.0,11380,은평구,Eunpyeong-gu,1.0,37.6176125,126.9227004
2,3.0,11230,동대문구,Dongdaemun-gu,2.0,37.5838012,127.0507003
3,4.0,11590,동작구,Dongjak-gu,3.0,37.4965037,126.9443073
4,5.0,11545,금천구,Geumcheon-gu,4.0,37.4600969,126.9001546


### 데이터프레임을 엑셀로 내보내기

In [231]:
seoul_sgg_df.to_excel("seoul_sgg_list.xlsx", index=False)

## 시군구별 인구통계 데이터 가져오기

In [244]:
url_args = [SEOUL_API_AUTH_KEY, "json", "octastatapi419", 1, 26, ""]

seoul_pop_list = seoul_open_api_data(url, url_args)
seoul_pop_df = pd.DataFrame(seoul_pop_list)

seoul_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GIGAN                  26 non-null     object
 1   JACHIGU                26 non-null     object
 2   SEDAE                  26 non-null     object
 3   GYE_1                  26 non-null     object
 4   NAMJA_1                26 non-null     object
 5   YEOJA_1                26 non-null     object
 6   GYE_2                  26 non-null     object
 7   NAMJA_2                26 non-null     object
 8   YEOJA_2                26 non-null     object
 9   GYE_3                  26 non-null     object
 10  NAMJA_3                26 non-null     object
 11  YEOJA_3                26 non-null     object
 12  SEDAEDANGINGU          26 non-null     object
 13  N_65SEISANGGORYEONGJA  26 non-null     object
dtypes: object(14)
memory usage: 3.0+ KB


In [245]:
seoul_pop_df.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,﻿2010.1-4,합계,"4,121,193","10,464,171","5,181,117","5,283,054","10,208,670","5,059,504","5,149,166","255,501","121,613","133,888",2.48,"956,414"
1,2010.1-4,종로구,"74,300","177,023","88,225","88,798","168,158","84,088","84,070","8,865","4,137","4,728",2.26,"20,925"
2,2010.1-4,중구,"58,472","138,102","69,175","68,927","129,786","65,216","64,570","8,316","3,959","4,357",2.22,"16,208"
3,2010.1-4,용산구,"107,407","250,844","123,295","127,549","238,366","116,764","121,602","12,478","6,531","5,947",2.22,"28,436"
4,2010.1-4,성동구,"126,942","316,113","158,228","157,885","308,144","154,389","153,755","7,969","3,839","4,130",2.43,"30,704"


### boolean 참조
* True 에 해당되는 데이터 추출

In [258]:
condition = seoul_pop_df['JACHIGU'] != "합계"
seoul_pop_df_selected = seoul_pop_df[condition]
seoul_pop_df_selected.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
1,2010.1-4,종로구,"74,300","177,023","88,225","88,798","168,158","84,088","84,070","8,865","4,137","4,728",2.26,"20,925"
2,2010.1-4,중구,"58,472","138,102","69,175","68,927","129,786","65,216","64,570","8,316","3,959","4,357",2.22,"16,208"
3,2010.1-4,용산구,"107,407","250,844","123,295","127,549","238,366","116,764","121,602","12,478","6,531","5,947",2.22,"28,436"
4,2010.1-4,성동구,"126,942","316,113","158,228","157,885","308,144","154,389","153,755","7,969","3,839","4,130",2.43,"30,704"
5,2010.1-4,광진구,"156,264","386,292","190,823","195,469","373,586","185,087","188,499","12,706","5,736","6,970",2.39,"30,765"


In [259]:
seoul_pop_df_selected[['JACHIGU', 'GYE_1']]

,JACHIGU,GYE_1
1,종로구,"177,023"
2,중구,"138,102"
3,용산구,"250,844"
4,성동구,"316,113"
5,광진구,"386,292"
6,동대문구,"374,039"
7,중랑구,"431,283"
8,성북구,"484,359"
9,강북구,"342,669"
10,도봉구,"371,095"


In [265]:
seoul_pop_df_final = seoul_pop_df_selected[['JACHIGU', 'GYE_1']]
seoul_pop_df_final.columns = ['시군구명', '주민등록인구']
seoul_pop_df_final.head()

,시군구명,주민등록인구
1,종로구,"177,023"
2,중구,"138,102"
3,용산구,"250,844"
4,성동구,"316,113"
5,광진구,"386,292"


In [267]:
seoul_pop_df_final.to_excel("sgg_pop.xlsx", index=False)

## 서울시 동별 사업체현황 api

In [269]:
url_args = [SEOUL_API_AUTH_KEY, "json", "octastatapi104", 1, 450, ""]
biz_data_list = seoul_open_api_data(url, url_args)
sgg_biz_df = pd.DataFrame(biz_data_list)
sgg_biz_df.head()

,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,SAEOPCHESU_2,JONGSAJASU_1,...,JONGSAJASU_4,SAEOPCHESU_6,JONGSAJASU_5,SAEOPCHESU_7,JONGSAJASU_6,SAEOPCHESU_8,JONGSAJASU_7,SAEOPCHESU_9,JONGSAJASU_8,SAEOPCHESU_10
0,2015,합계,합계,820658,272567,5108828,2887719,2221109,17,491,...,8713,463,6712,22695,360192,234494,912406,93642,279921,126785
1,2015,종로구,소계,40871,13472,269106,152766,116340,1,16,...,387,9,179,358,25269,17447,46970,1418,7841,6635
2,2015,종로구,사직동,3551,1092,51425,29781,21644,1,16,...,2,2,32,38,4158,563,4074,280,2281,856
3,2015,종로구,삼청동,738,354,4736,2374,2362,-,-,...,-,-,-,6,15,273,864,20,84,261
4,2015,종로구,부암동,540,224,3635,1714,1921,-,-,...,-,-,-,22,53,104,523,33,37,125


In [272]:
sgg_biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   GIGAN             450 non-null    object
 1   JACHIGU           450 non-null    object
 2   DONG              450 non-null    object
 3   SAEOPCHESU_1      450 non-null    object
 4   YEOSEONGDAEPYOJA  450 non-null    object
 5   GYE               450 non-null    object
 6   NAM               450 non-null    object
 7   YEO               450 non-null    object
 8   SAEOPCHESU_2      450 non-null    object
 9   JONGSAJASU_1      450 non-null    object
 10  SAEOPCHESU_3      450 non-null    object
 11  JONGSAJASU_2      450 non-null    object
 12  SAEOPCHESU_4      450 non-null    object
 13  JONGSAJASU_3      450 non-null    object
 14  SAEOPCHESU_5      450 non-null    object
 15  JONGSAJASU_4      450 non-null    object
 16  SAEOPCHESU_6      450 non-null    object
 17  JONGSAJASU_5    

In [277]:
# 소계 == 구 단위 사업체 현황 총합
sgg_biz_df_selected = sgg_biz_df[sgg_biz_df['DONG'] == '소계']

In [278]:
sgg_biz_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 431
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   GIGAN             25 non-null     object
 1   JACHIGU           25 non-null     object
 2   DONG              25 non-null     object
 3   SAEOPCHESU_1      25 non-null     object
 4   YEOSEONGDAEPYOJA  25 non-null     object
 5   GYE               25 non-null     object
 6   NAM               25 non-null     object
 7   YEO               25 non-null     object
 8   SAEOPCHESU_2      25 non-null     object
 9   JONGSAJASU_1      25 non-null     object
 10  SAEOPCHESU_3      25 non-null     object
 11  JONGSAJASU_2      25 non-null     object
 12  SAEOPCHESU_4      25 non-null     object
 13  JONGSAJASU_3      25 non-null     object
 14  SAEOPCHESU_5      25 non-null     object
 15  JONGSAJASU_4      25 non-null     object
 16  SAEOPCHESU_6      25 non-null     object
 17  JONGSAJASU_5     

In [295]:
sgg_biz_df_final = sgg_biz_df_selected[['JACHIGU', 'GYE', 'SAEOPCHESU_1']]
sgg_biz_df_final.columns = ['시군구명', '종사자수', '사업체수']

### 인덱스 번호 초기화

In [296]:
sgg_biz_df_final.reset_index(drop=True)

,시군구명,종사자수,사업체수
0,종로구,269106,40871
1,중구,423808,66190
2,용산구,133446,21178
3,성동구,162019,26130
4,광진구,123689,24531
5,동대문구,143858,32994
6,중랑구,99241,27287
7,성북구,113893,24690
8,강북구,69787,18654
9,도봉구,68669,18455


In [297]:
sgg_biz_df_final.to_excel("sgg_biz.xlsx", index=False)

In [304]:
seoul_starbucks_df = pd.read_excel("seoul_starbucks_list.xlsx")
seoul_starbucks_df.head()

,STORE,STORE_TYPE,PHONE_NUMBER,LAT,LNG,ADDR
0,역삼아레나빌딩,general,1522-3232,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동)
1,논현역사거리,general,1522-3232,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동)
2,신사역성일빌딩,general,1522-3232,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동)
3,국기원사거리,general,1522-3232,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동)
4,스탈릿대치R,reserve,1522-3232,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동)


In [306]:
seoul_starbucks_df['ADDR']

0                서울특별시 강남구 언주로 425 (역삼동)
1               서울특별시 강남구 강남대로 538 (논현동)
2               서울특별시 강남구 강남대로 584 (논현동)
3               서울특별시 강남구 테헤란로 125 (역삼동)
4             서울특별시 강남구 남부순환로 2947 (대치동)
                     ...                
517                     서울특별시 중랑구 신내로 72
518                    서울특별시 중랑구 면목로 310
519      서울특별시 중랑구 망우로 307, ,3,4번지 (상봉동)
520    서울특별시 중랑구 동일로 932, 묵동이마트 B1층 (묵동)
521                       서울 중랑구 동일로 952
Name: ADDR, Length: 522, dtype: object

## 문자열 분리

In [311]:
sgg_names = []
for addr in seoul_starbucks_df['ADDR']:
    sgg = addr.split()[1]
    sgg_names.append(sgg)
    
seoul_starbucks_df['시군구명']=sgg_names
seoul_starbucks_df.head()

,STORE,STORE_TYPE,PHONE_NUMBER,LAT,LNG,ADDR,시군구명
0,역삼아레나빌딩,general,1522-3232,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),강남구
1,논현역사거리,general,1522-3232,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),강남구
2,신사역성일빌딩,general,1522-3232,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),강남구
3,국기원사거리,general,1522-3232,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),강남구
4,스탈릿대치R,reserve,1522-3232,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),강남구


In [327]:
seoul_starbucks_df[seoul_starbucks_df['시군구명'] == '67'] = '금천구'
seoul_starbucks_df.to_excel("seoul_starbucks_list.xlsx", index=False)

## 데이터 병합

* 피봇 함수(pivot, pivot_table): 피봇팅(특정 기준에 따른 데이터 요약)
* merge() 함수
* 예제
|시군구 명|스타벅스 매장수|
|:-:|:-:|
|강남구|100|
|...|...|

In [360]:
seoul_sgg = pd.read_excel("seoul_sgg_list.xlsx")

In [361]:
seoul_starbucks = pd.read_excel("seoul_starbucks_list.xlsx")

In [362]:
seoul_starbucks

,STORE,STORE_TYPE,PHONE_NUMBER,LAT,LNG,ADDR,시군구명
0,역삼아레나빌딩,general,1522-3232,37.5011,127.043,서울특별시 강남구 언주로 425 (역삼동),강남구
1,논현역사거리,general,1522-3232,37.5102,127.022,서울특별시 강남구 강남대로 538 (논현동),강남구
2,신사역성일빌딩,general,1522-3232,37.5141,127.021,서울특별시 강남구 강남대로 584 (논현동),강남구
3,국기원사거리,general,1522-3232,37.4995,127.031,서울특별시 강남구 테헤란로 125 (역삼동),강남구
4,스탈릿대치R,reserve,1522-3232,37.4947,127.063,서울특별시 강남구 남부순환로 2947 (대치동),강남구
...,...,...,...,...,...,...,...
517,중랑구청,general,1522-3232,37.6054,127.096,서울특별시 중랑구 신내로 72,중랑구
518,사가정역,general,1522-3232,37.5796,127.088,서울특별시 중랑구 면목로 310,중랑구
519,상봉역,general,1522-3232,37.5969,127.086,"서울특별시 중랑구 망우로 307, ,3,4번지 (상봉동)",중랑구
520,묵동이마트,general,1522-3232,37.6134,127.077,"서울특별시 중랑구 동일로 932, 묵동이마트 B1층 (묵동)",중랑구


In [363]:
starbucks_sgg_count = seoul_starbucks.pivot_table(values='STORE', index='시군구명', aggfunc='count').rename(columns={'STORE': '스타벅스_매장수'})

In [364]:
seoul_sgg.rename(columns={'SIG_KOR_NM': '시군구명'}, inplace=True)

In [365]:
# 병합
seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how='left', on='시군구명') #왼쪽이 기준되어 merge
seoul_sgg.head()

,OBJECTID,SIG_CD,시군구명,SIG_ENG_NM,ESRI_PK,LAT,LNG,스타벅스_매장수
0,1,11320,도봉구,Dobong-gu,0,37.665861,127.031767,2
1,2,11380,은평구,Eunpyeong-gu,1,37.617612,126.922700,8
2,3,11230,동대문구,Dongdaemun-gu,2,37.583801,127.050700,8
3,4,11590,동작구,Dongjak-gu,3,37.496504,126.944307,11
4,5,11545,금천구,Geumcheon-gu,4,37.460097,126.900155,11


In [366]:
seoul_sgg_pop = pd.read_excel("sgg_pop.xlsx")
seoul_sgg_pop.head()

,시군구명,주민등록인구
0,종로구,"177,023"
1,중구,"138,102"
2,용산구,"250,844"
3,성동구,"316,113"
4,광진구,"386,292"


In [367]:
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_pop, how='left', on='시군구명')

In [371]:
seoul_sgg.head()

,OBJECTID,SIG_CD,시군구명,SIG_ENG_NM,ESRI_PK,LAT,LNG,스타벅스_매장수,주민등록인구
0,1,11320,도봉구,Dobong-gu,0,37.665861,127.031767,2,"371,095"
1,2,11380,은평구,Eunpyeong-gu,1,37.617612,126.922700,8,"474,165"
2,3,11230,동대문구,Dongdaemun-gu,2,37.583801,127.050700,8,"374,039"
3,4,11590,동작구,Dongjak-gu,3,37.496504,126.944307,11,"407,966"
4,5,11545,금천구,Geumcheon-gu,4,37.460097,126.900155,11,"262,337"


In [374]:
sgg_biz = pd.read_excel("sgg_biz.xlsx")
sgg_biz.head()

,시군구명,종사자수,사업체수
0,종로구,269106,40871
1,중구,423808,66190
2,용산구,133446,21178
3,성동구,162019,26130
4,광진구,123689,24531


In [375]:
seoul_sgg = pd.merge(seoul_sgg, sgg_biz, how='left', on='시군구명')

In [376]:
seoul_sgg.head()

,OBJECTID,SIG_CD,시군구명,SIG_ENG_NM,ESRI_PK,LAT,LNG,스타벅스_매장수,주민등록인구,종사자수,사업체수
0,1,11320,도봉구,Dobong-gu,0,37.665861,127.031767,2,"371,095",68669,18455
1,2,11380,은평구,Eunpyeong-gu,1,37.617612,126.922700,8,"474,165",87693,24179
2,3,11230,동대문구,Dongdaemun-gu,2,37.583801,127.050700,8,"374,039",143858,32994
3,4,11590,동작구,Dongjak-gu,3,37.496504,126.944307,11,"407,966",103915,19609
4,5,11545,금천구,Geumcheon-gu,4,37.460097,126.900155,11,"262,337",223058,30080


In [379]:
seoul_sgg.to_excel("seoul_sgg_stat.xlsx", index=False)

In [380]:
seoul_sgg.sort_values(by=['스타벅스_매장수'], ascending=False).head()

,OBJECTID,SIG_CD,시군구명,SIG_ENG_NM,ESRI_PK,LAT,LNG,스타벅스_매장수,주민등록인구,종사자수,사업체수
9,10,11680,강남구,Gangnam-gu,9,37.495985,127.066409,80,"570,913",711278,73590
11,12,11140,중구,Jung-gu,11,37.557945,126.994190,52,"138,102",423808,66190
15,16,11650,서초구,Seocho-gu,16,37.476953,127.037810,46,"432,888",439963,47061
6,7,11110,종로구,Jongno-gu,6,37.599100,126.986149,38,"177,023",269106,40871
21,22,11560,영등포구,Yeongdeungpo-gu,23,37.520641,126.913924,34,"440,863",362524,44512


In [381]:
seoul_sgg[['종사자수', '사업체수']] = seoul_sgg[['종사자수', '사업체수']].astype(int)

In [382]:
seoul_sgg.dtypes

OBJECTID        int64
SIG_CD          int64
시군구명           object
SIG_ENG_NM     object
ESRI_PK         int64
LAT           float64
LNG           float64
스타벅스_매장수        int64
주민등록인구         object
종사자수            int64
사업체수            int64
dtype: object

In [384]:
seoul_sgg['주민등록인구'] = seoul_sgg['주민등록인구'].str.replace(',','').astype(int)
seoul_sgg.dtypes

OBJECTID        int64
SIG_CD          int64
시군구명           object
SIG_ENG_NM     object
ESRI_PK         int64
LAT           float64
LNG           float64
스타벅스_매장수        int64
주민등록인구          int64
종사자수            int64
사업체수            int64
dtype: object

In [388]:
seoul_sgg.sort_values(by=['스타벅스_매장수'], ascending=False, inplace=True)

In [390]:
seoul_sgg.reset_index(drop=True)

,OBJECTID,SIG_CD,시군구명,SIG_ENG_NM,ESRI_PK,LAT,LNG,스타벅스_매장수,주민등록인구,종사자수,사업체수
0,10,11680,강남구,Gangnam-gu,9,37.495985,127.066409,80,570913,711278,73590
1,12,11140,중구,Jung-gu,11,37.557945,126.994190,52,138102,423808,66190
2,16,11650,서초구,Seocho-gu,16,37.476953,127.037810,46,432888,439963,47061
3,7,11110,종로구,Jongno-gu,6,37.599100,126.986149,38,177023,269106,40871
4,22,11560,영등포구,Yeongdeungpo-gu,23,37.520641,126.913924,34,440863,362524,44512
5,15,11440,마포구,Mapo-gu,14,37.562291,126.908780,32,392087,252534,36504
6,19,11710,송파구,Songpa-gu,19,37.504853,127.114482,31,689909,302517,45375
7,25,11170,용산구,Yongsan-gu,24,37.531101,126.981074,19,250844,133446,21178
8,20,11410,서대문구,Seodaemun-gu,21,37.582037,126.935667,19,336490,111615,19612
9,11,11500,강서구,Gangseo-gu,10,37.565762,126.822656,16,578257,199289,33273


In [391]:
seoul_sgg.to_excel("seoul_sgg_stat.xlsx", index=False)